# Download Data

In [7]:
import requests
import pandas as pd
from io import BytesIO

def download_csv(csv_url):
    response = requests.get(csv_url)
    response.raise_for_status()

    try:
        df = pd.read_csv(BytesIO(response.content), sep=',', low_memory=False)
    except pd.errors.ParserError:
        df = pd.read_csv(BytesIO(response.content), sep=';', error_bad_lines=False)

    return df

In [8]:
recipes_df = download_csv("https://media.githubusercontent.com/media/DonatoFe11/SustainaMeal_Case_Study/master/data/final_recipes_set.csv")

In [5]:
recipes_df

,title,sustainability_score,sustainability_label,Unnamed: 0,recipe_id,description,author_id,duration,directions,ingredients,...,protein [g],direction_size,ingredients_sizes,who_score,fsa_score,nutri_score,normalization_comment,ingredient_food_kg_urls,ingredient_food_kg_names,healthiness_label
0,Boiled Radishes,0.0025019511128497,0,294268,179840,These taste a lot like new potatoes. The boil...,254469,22.0,"['Trim ends and bad spots off Radishes.', 'Boi...","['radishes size bag bunch', 'water radishes']",...,0.8,2.0,2,0.293040293040293,1.000,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['applesauce', 'garlic clove', 'ginger', 'mola...",0.0
1,Apple Cider Reduction,0.0026229519800712,0,59722,521756,I saw a post about this on Facebook and gave i...,171084,155.0,"['In 6-8 quart stock pan, measure out 2 cups o...","['gallon apple cider fresh', 'gallon apple cid...",...,0.0,10.0,2,0.2142857142857142,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['bean sprouts', 'chicken thighs', 'fresh ging...",1.0
2,Homemade Apple Pectin Stock,0.0026229519800712,0,159910,25188,Easy to make and works well.,10404,25.0,['Slice unpeeled apples including cores and se...,"['tart apple', 'water pound apples']",...,1.2,11.0,2,0.2784138818621577,0.750,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['baking powder', 'baking soda', 'cinnamon', '...",0.0
3,Easy Applesauce,0.0026229519800712,0,421226,264179,Use fresh cooking apples for this for best res...,37636,50.0,['Place apples and cider into a large saucepan...,"['apples', 'apple cider']",...,0.5,3.0,2,0.2765246449456975,0.875,0.75,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['carrots', 'fresh peas', 'fresh tomatoes', 'g...",0.0
4,Cider Baked Sausage,0.0027519668838767,0,374088,101966,A very easy to make main dish perfect for fall...,30534,50.0,"['Preheat oven to 350°.', 'Prick sausages with...","['breakfast sausage links', 'apples Smith', 'o...",...,46.4,4.0,4,0.1467136150234742,0.125,0.00,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Shedd Spread Country Crock', 'cayenne pepper...",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93826,Curried Lamb on Rice,0.9510240380047836,2,103145,373244,"With modification this recipe is from ""The Dia...",133174,65.0,"['Over medium heat, melt the butter in a large...","['lamb', 'medium onion', 'cloves', 'butter', '...",...,28.7,5.0,13,0.2235485975212003,0.625,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['Dijon mustard', 'Polish sausage', 'apricot p...",1.0
93827,Rich Lamb Curry,0.9605043058827348,2,442694,268247,The recipe isn't as time consuming as it looks...,491437,110.0,"['Heat 1 tbsp olive oil in dutch oven. ', 'Co...","['oil', 'lamb', 'flour', 'salt pepper', 'onion...",...,36.1,9.0,21,0.1533283470456245,0.000,0.00,NaN,NaN,NaN,1.0
93828,"Middle Eastern Slow-Cooked Stew With Lamb, Chi...",0.96845815092894,2,359960,144850,From Cooking Light. Per 3/4 c. serving: 310 ca...,37779,112.0,['Let the oil get heating in a large pot over ...,"['oil', 'lamb', 'onions rings', 'water', 'clov...",...,20.8,12.0,19,0.2055658627087198,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"['anchovies', 'celery ribs', 'diced tomatoes',...",1.0
93829,Five Meat Chili Con Carne With Beans,0.9714538116434204,2,37637,13568,yum,20571,210.0,"['In a large pot brown meat in stages, about a...","['beef', 'pork', 'lamb', 'sausage', 'sausage',...",...,14.5,7.0,15,0.1975683890577508,0.250,0.25,NaN,['http://idea.rpi.edu/heals/kb/ingredientname/...,"[""French\\'s French fried onions"", 'crabmeat',...",1.0


# SustainaMeal

Initializes the system by loading the data and preparing the embeddings.

In [6]:
from HeASe.sustainameal import SustainaMeal

nutrients = ['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]']

sm = SustainaMeal(
    recipes_df=recipes_df,
    nutrients=['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]'],
    transformer_name='davanstrien/autotrain-recipes-2451975973'
)

/home/festa/SustainaMeal_Case_Study/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: unsupported operand type(s) for -: 'int' and 'str'

# Upload Recipes of Experiments

In [16]:
import pandas as pd

# Paths to the JSON files

sustainability_label_2_recipes_file = '../experiments/data/sustainability_label_2_recipes.json'

# Read the JSON files into pandas DataFrames

sustainability_label_2_recipes = pd.read_json(sustainability_label_2_recipes_file)


# Compare LLM Rerank

In [17]:
sm.setup_llama_model_id()
sm.create_agent()

Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.01s/it]


In [18]:
acceptable_tags = ['appetizers', 'main-dish', 'side-dishes', 'drinks', 'beverages', 'fruits', 'desserts',
                   'breakfast', 'pasta-rice-and-grains', 'rice', 'pasta', 'pizza', 'breads', 'meat', 'fish',
                   'seafood', 'beef', 'chicken', 'vegetarian']

In [21]:
nutrients_string = ", ".join(nutrients)
print(nutrients_string)

calories [cal], totalFat [g], saturatedFat [g], cholesterol [mg], sodium [mg], dietaryFiber [g], sugars [g], protein [g]


In [22]:
selected_recipes = pd.concat([sustainability_label_2_recipes])

In [23]:
len(selected_recipes)

100

In [24]:
# Definisci le soglie
WHO_SCORE_THRESHOLDS = ( 0.15, 0.25)  # (soglia media, soglia alta)
SUSTAINABILITY_SCORE_THRESHOLDS = (0.1, 0.5)  # (soglia media, soglia alta)

# Funzioni di aiuto per calcolare le label
def calculate_healthiness_label(who_score):
    if who_score > WHO_SCORE_THRESHOLDS[1]:
        return "0"
    elif who_score < WHO_SCORE_THRESHOLDS[0]:
        return "2"
    else:
        return "1"

def calculate_sustainability_label(sustainability_score):
    if sustainability_score <= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "0"
    elif sustainability_score >= SUSTAINABILITY_SCORE_THRESHOLDS[1]:
        return "2"
    else:
        return "1"

# Calcola le label in base ai valori di who_score e sustainability_score

In [25]:
similar_recipes = sm.find_similar_recipes("Pasta", 10, acceptable_tags=acceptable_tags, match_all_tags=False)

Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 58.18batch/s]

Tags to match: ['main-dish', 'pasta', 'vegetarian', 'pasta-rice-and-grains']
Number of filtered recipes: 43799


In [26]:
order_recipes = sm.order_recipe_by_sustainameal()
print(order_recipes)

                                                   title  similarity_score  \
15954                             Brown Rice and Edamame          0.983306   
16511  Tofu With Red Curry Paste, Peas, and Yellow To...          0.986884   
11836                Pasta with Tomatoes, Peas and Cream          0.989764   
29501                                          Tofu Bowl          0.982852   
50905                       Asparagus-Mushroom Primavera          0.988871   
64425         Angel Hair With Tomatoes, Basil and Garlic          0.983267   
70774                  Spaghettini With Smothered Onions          0.983303   
65074                    Orlando Bloom's Pasta Au Pistou          0.985402   
73434               Mini Farfalle With Tomatoes and Corn          0.991559   
85469                Pasta and Potatoes (Pasta e Patate)          0.984727   

       who_score  healthiness_increment  sustainability_score  \
15954   0.345328              47.711098              0.909384   
16511   0.3

In [27]:
# non ne capisco il senso
order_recipes_llama = sm.order_recipe_by_sustainameal()
print(order_recipes_llama)

                                                   title  similarity_score  \
15954                             Brown Rice and Edamame          0.983306   
16511  Tofu With Red Curry Paste, Peas, and Yellow To...          0.986884   
11836                Pasta with Tomatoes, Peas and Cream          0.989764   
29501                                          Tofu Bowl          0.982852   
50905                       Asparagus-Mushroom Primavera          0.988871   
64425         Angel Hair With Tomatoes, Basil and Garlic          0.983267   
70774                  Spaghettini With Smothered Onions          0.983303   
65074                    Orlando Bloom's Pasta Au Pistou          0.985402   
73434               Mini Farfalle With Tomatoes and Corn          0.991559   
85469                Pasta and Potatoes (Pasta e Patate)          0.984727   

       who_score  healthiness_increment  sustainability_score  \
15954   0.345328              47.711098              0.909384   
16511   0.3

In [28]:
best_choice_by_llama = sm.choose_best_recipe_with_llama()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [29]:
print(best_choice_by_llama)

Let's dive into the delicious and nutritious Brown Rice and Edamame recipe. This dish is a fantastic choice for a healthy and sustainable meal. Brown rice is a whole grain that's rich in fiber, vitamins, and minerals, making it an excellent source of sustained energy. Edamame, on the other hand, are high in protein and fiber, and are also packed with vitamins and minerals like calcium and iron. This recipe is also incredibly versatile, allowing you to customize it with your favorite seasonings and herbs. Plus, brown rice and edamame are both relatively low-cost and accessible ingredients, making this recipe a great option for a budget-friendly and sustainable meal. By choosing this recipe, you'll be nourishing your body with wholesome ingredients while also reducing your environmental impact.
